In [ ]:
# ===============================================================
# PROJETO ENANI - OTIMIZAÇÃO DE MODELOS PARA PREDIÇÃO NUTRICIONAL
# ===============================================================
# Arquivo: model_tuning_optimization.ipynb
# Objetivo: Tunar os 3 melhores modelos identificados no screening
# Dataset: TrainDS.csv (dados de treino)
# ===============================================================

import pandas as pd
import numpy as np
from pycaret.classification import *
import warnings
warnings.filterwarnings('ignore')

# ===============================================================
# 1. INTRODUÇÃO E CONTEXTO DO PROJETO
# ===============================================================

print("🚀 PROJETO ENANI - OTIMIZAÇÃO DE MODELOS")
print("=" * 60)
print("📋 OBJETIVO: Otimizar modelos para predição de status nutricional")
print("🎯 META: Melhorar AUC de 0.55 para >0.70 e F1 de 0.59 para >0.75")
print("📊 MODELOS SELECIONADOS: XGBoost, Gradient Boosting, Random Forest")
print()

# ===============================================================
# 2. CARREGAR E VERIFICAR DADOS
# ===============================================================

print("📁 CARREGANDO DATASET DE TREINO")
print("=" * 60)

# Caminho do dataset
train_path = '/Users/marcelosilva/Desktop/projectOne/5/A-TrainTestDataset/TrainDS.csv'
df_train = pd.read_csv(train_path)

print(f"✅ Dataset carregado: {df_train.shape}")
print(f"📊 Linhas: {df_train.shape[0]:,}")
print(f"📊 Colunas: {df_train.shape[1]}")

# ===============================================================
# 3. DOCUMENTAÇÃO DOS LABEL ENCODINGS APLICADOS
# ===============================================================

print(f"\n📋 LABEL ENCODINGS APLICADOS NO DATASET")
print("=" * 60)

print("🎯 TARGET VARIABLE (status_nutricional_who):")
print("   0 = Desnutrido (mais grave)")
print("   1 = Peso adequado (ideal)")  
print("   2 = Sobrepeso (excesso leve)")
print("   3 = Obesidade (excesso severo)")

print("\n📊 VARIÁVEIS ORDINAIS (Label Encoded):")
print("   def_idade_gest: 0=prematuro, 1=adequado, 2=pos_termo")
print("   adequacao_prenatal: 0=ausente, 1=insuficiente, 2=adequado")
print("   idade_mae_cat: 0=jovem, 1=adulta, 2=madura")
print("   peso_cat: 0=baixo, 1=normal, 2=alto")
print("   classificacao_peso: 0=PIG, 1=AIG, 2=GIG")

print("\n🎭 VARIÁVEIS CATEGÓRICAS (One-Hot pelo PyCaret):")
categorical_vars = ['b02_sexo', 'd01_cor', 'h04_parto', 'j03_cor', 'k15_recebeu', 'k16_liquido']
for var in categorical_vars:
    print(f"   {var}: Receberá One-Hot encoding automático")

# ===============================================================
# 4. ANÁLISE DA DISTRIBUIÇÃO DO TARGET
# ===============================================================

print(f"\n🎯 DISTRIBUIÇÃO DO TARGET (CRITICAL FOR TUNING)")
print("=" * 60)

target_dist = df_train['status_nutricional_who'].value_counts().sort_index()
target_names = ['Desnutrido', 'Peso adequado', 'Sobrepeso', 'Obesidade']

total_samples = len(df_train)
for value, count in target_dist.items():
    percentage = (count / total_samples) * 100
    status = "⚠️ MINORITÁRIA" if percentage < 5 else "✅"
    print(f"   Classe {value} ({target_names[value]}): {count:,} ({percentage:.1f}%) {status}")

# Verificar desbalanceamento crítico
minority_class_pct = (target_dist[0] / total_samples) * 100
if minority_class_pct < 5:
    print(f"\n🚨 ALERTA: Classe minoritária com {minority_class_pct:.1f}% - SMOTE será crítico!")

# ===============================================================
# 5. CONFIGURAÇÃO DO PYCARET PARA OTIMIZAÇÃO
# ===============================================================

print(f"\n🔧 CONFIGURANDO PYCARET PARA OTIMIZAÇÃO")
print("=" * 60)

# Verificar memória disponível
import psutil
memory_gb = psutil.virtual_memory().available / (1024**3)
print(f"💾 Memória disponível: {memory_gb:.1f} GB")

# Features a ignorar
ignore_features = ['id_anon', 'vd_zimc']

# Variáveis ordinais (CRÍTICO para preservar ordem)
ordinal_features = {
    'def_idade_gest': [0, 1, 2],           # prematuro < adequado < pos_termo
    'adequacao_prenatal': [0, 1, 2],       # ausente < insuficiente < adequado
    'idade_mae_cat': [0, 1, 2],            # jovem < adulta < madura
    'peso_cat': [0, 1, 2],                 # baixo < normal < alto
    'classificacao_peso': [0, 1, 2]        # PIG < AIG < GIG
}

print(f"🚫 Features ignoradas: {ignore_features}")
print(f"📊 Variáveis ordinais configuradas: {len(ordinal_features)}")

try:
    # Setup otimizado para tuning
    clf = setup(
        data=df_train,
        target='status_nutricional_who',
        
        # Features e encoding
        ignore_features=ignore_features,
        ordinal_features=ordinal_features,
        
        # Balanceamento (CRÍTICO para classe minoritária)
        fix_imbalance=True,
        fix_imbalance_method='smote',
        
        # Preprocessing otimizado
        remove_multicollinearity=True,
        multicollinearity_threshold=0.95,
        feature_selection=False,  # Manter todas para tuning
        transformation=True,
        normalize=True,
        
        # Validação robusta
        fold=5,  # 5-fold para tuning (mais rápido que 10)
        train_size=0.8,  # 80% treino, 20% validação interna
        
        # Performance
        session_id=123,
        use_gpu=False,
        n_jobs=-1
    )
    
    print("✅ SETUP OTIMIZADO CONCLUÍDO!")
    
    # Verificar configuração aplicada
    print(f"\n📊 CONFIGURAÇÃO APLICADA:")
    print(f"   Shape após transformações: {get_config('X_train').shape}")
    print(f"   SMOTE aplicado: Sim")
    print(f"   Folds para CV: 5")
    print(f"   Variáveis ordinais: {len(ordinal_features)}")
    
except Exception as e:
    print(f"❌ ERRO no setup: {e}")
    exit()

# ===============================================================
# 6. BASELINE - MODELOS IDENTIFICADOS NO SCREENING
# ===============================================================

print(f"\n📊 BASELINE - RESULTADOS DO SCREENING ANTERIOR")
print("=" * 60)

baseline_results = {
    'XGBoost': {'F1': 0.592, 'AUC': 0.540, 'Accuracy': 0.626},
    'Gradient Boosting': {'F1': 0.591, 'AUC': 0.556, 'Accuracy': 0.614},
    'Random Forest': {'F1': 0.571, 'AUC': 0.539, 'Accuracy': 0.560}
}

print("RESULTADOS DO SCREENING (sem otimização):")
for model, metrics in baseline_results.items():
    print(f"   {model}:")
    print(f"      F1: {metrics['F1']:.3f} | AUC: {metrics['AUC']:.3f} | Accuracy: {metrics['Accuracy']:.3f}")

print(f"\n🎯 METAS DE MELHORIA:")
print(f"   AUC: 0.540 → >0.700 (+30%)")
print(f"   F1:  0.592 → >0.750 (+27%)")

# ===============================================================
# 7. OTIMIZAÇÃO DO XGBOOST (MELHOR F1 BASELINE)
# ===============================================================

print(f"\n🚀 OTIMIZANDO XGBOOST (Prioridade: AUC)")
print("=" * 60)

try:
    print("⏳ Iniciando otimização XGBoost (pode demorar 3-5 minutos)...")
    
    # Criar modelo base
    xgb_base = create_model('xgboost', verbose=False)
    
    # Otimização focada em AUC (problema principal) - SEM OPTUNA
    xgb_tuned = tune_model(
        xgb_base,
        optimize='AUC',           # PRIORIDADE: melhorar discriminação
        n_iter=50,                # 50 iterações (sem optuna)
        fold=5,                   # 5-fold para velocidade
        verbose=False
    )
    
    print("✅ XGBoost otimizado com sucesso!")
    
    # Avaliar modelo otimizado
    print("\n📊 AVALIANDO XGBOOST OTIMIZADO:")
    xgb_results = pull()  # Pegar resultados da última operação
    
    print(f"   AUC otimizado: {xgb_results['AUC'].mean():.3f} (baseline: 0.540)")
    print(f"   F1 otimizado:  {xgb_results['F1'].mean():.3f} (baseline: 0.592)")
    print(f"   Accuracy:      {xgb_results['Accuracy'].mean():.3f} (baseline: 0.626)")
    
    # Calcular melhoria
    auc_improvement = ((xgb_results['AUC'].mean() - 0.540) / 0.540) * 100
    f1_improvement = ((xgb_results['F1'].mean() - 0.592) / 0.592) * 100
    
    print(f"   🚀 Melhoria AUC: {auc_improvement:+.1f}%")
    print(f"   🚀 Melhoria F1:  {f1_improvement:+.1f}%")
    
except Exception as e:
    print(f"❌ ERRO na otimização XGBoost: {e}")
    xgb_tuned = None

# ===============================================================
# 8. OTIMIZAÇÃO DO GRADIENT BOOSTING (MELHOR AUC BASELINE)
# ===============================================================

print(f"\n🚀 OTIMIZANDO GRADIENT BOOSTING (Prioridade: F1)")
print("=" * 60)

try:
    print("⏳ Iniciando otimização Gradient Boosting...")
    
    # Criar modelo base
    gbc_base = create_model('gbc', verbose=False)
    
    # Otimização focada em F1 (dados desbalanceados) - SEM OPTUNA
    gbc_tuned = tune_model(
        gbc_base,
        optimize='F1',            # PRIORIDADE: dados desbalanceados
        n_iter=50,                # 50 iterações
        fold=5,
        verbose=False
    )
    
    print("✅ Gradient Boosting otimizado com sucesso!")
    
    # Avaliar modelo otimizado
    print("\n📊 AVALIANDO GRADIENT BOOSTING OTIMIZADO:")
    gbc_results = pull()
    
    print(f"   AUC otimizado: {gbc_results['AUC'].mean():.3f} (baseline: 0.556)")
    print(f"   F1 otimizado:  {gbc_results['F1'].mean():.3f} (baseline: 0.591)")
    print(f"   Accuracy:      {gbc_results['Accuracy'].mean():.3f} (baseline: 0.614)")
    
    # Calcular melhoria
    auc_improvement = ((gbc_results['AUC'].mean() - 0.556) / 0.556) * 100
    f1_improvement = ((gbc_results['F1'].mean() - 0.591) / 0.591) * 100
    
    print(f"   🚀 Melhoria AUC: {auc_improvement:+.1f}%")
    print(f"   🚀 Melhoria F1:  {f1_improvement:+.1f}%")
    
except Exception as e:
    print(f"❌ ERRO na otimização Gradient Boosting: {e}")
    gbc_tuned = None

# ===============================================================
# 9. OTIMIZAÇÃO DO RANDOM FOREST (BACKUP + INTERPRETABILIDADE)
# ===============================================================

print(f"\n🚀 OTIMIZANDO RANDOM FOREST (Prioridade: Recall)")
print("=" * 60)

try:
    print("⏳ Iniciando otimização Random Forest...")
    
    # Criar modelo base
    rf_base = create_model('rf', verbose=False)
    
    # Otimização focada em Recall (não perder casos críticos) - SEM OPTUNA
    rf_tuned = tune_model(
        rf_base,
        optimize='Recall',        # PRIORIDADE: encontrar todos os casos
        n_iter=30,                # 30 iterações (RF é mais simples)
        fold=5,
        verbose=False
    )
    
    print("✅ Random Forest otimizado com sucesso!")
    
    # Avaliar modelo otimizado
    print("\n📊 AVALIANDO RANDOM FOREST OTIMIZADO:")
    rf_results = pull()
    
    print(f"   AUC otimizado: {rf_results['AUC'].mean():.3f} (baseline: 0.539)")
    print(f"   F1 otimizado:  {rf_results['F1'].mean():.3f} (baseline: 0.571)")
    print(f"   Recall:        {rf_results['Recall'].mean():.3f}")
    
    # Calcular melhoria
    auc_improvement = ((rf_results['AUC'].mean() - 0.539) / 0.539) * 100
    f1_improvement = ((rf_results['F1'].mean() - 0.571) / 0.571) * 100
    
    print(f"   🚀 Melhoria AUC: {auc_improvement:+.1f}%")
    print(f"   🚀 Melhoria F1:  {f1_improvement:+.1f}%")
    
except Exception as e:
    print(f"❌ ERRO na otimização Random Forest: {e}")
    rf_tuned = None

# ===============================================================
# 10. ENSEMBLE DOS MODELOS OTIMIZADOS
# ===============================================================

print(f"\n🎯 CRIANDO ENSEMBLE DOS MODELOS OTIMIZADOS")
print("=" * 60)

# Lista de modelos otimizados disponíveis
tuned_models = []
model_names = []

if xgb_tuned is not None:
    tuned_models.append(xgb_tuned)
    model_names.append("XGBoost")
    
if gbc_tuned is not None:
    tuned_models.append(gbc_tuned)
    model_names.append("Gradient Boosting")
    
if rf_tuned is not None:
    tuned_models.append(rf_tuned)
    model_names.append("Random Forest")

if len(tuned_models) >= 2:
    try:
        print(f"⏳ Criando ensemble de {len(tuned_models)} modelos...")
        
        # Ensemble por votação
        ensemble_voting = ensemble_model(
            tuned_models,
            method='Voting',
            optimize='AUC'
        )
        
        print("✅ Ensemble criado com sucesso!")
        
        # Avaliar ensemble
        print("\n📊 AVALIANDO ENSEMBLE:")
        ensemble_results = pull()
        
        print(f"   AUC ensemble: {ensemble_results['AUC'].mean():.3f}")
        print(f"   F1 ensemble:  {ensemble_results['F1'].mean():.3f}")
        print(f"   Accuracy:     {ensemble_results['Accuracy'].mean():.3f}")
        
    except Exception as e:
        print(f"❌ ERRO na criação do ensemble: {e}")
        ensemble_voting = None
else:
    print("⚠️ Não foi possível criar ensemble (menos de 2 modelos otimizados)")
    ensemble_voting = None

# ===============================================================
# 11. COMPARAÇÃO FINAL DOS RESULTADOS
# ===============================================================

print(f"\n📊 COMPARAÇÃO FINAL: BASELINE vs OTIMIZADO")
print("=" * 70)

print("BASELINE (Screening sem otimização):")
print("   XGBoost:          F1=0.592 | AUC=0.540 | Acc=0.626")
print("   Gradient Boost:   F1=0.591 | AUC=0.556 | Acc=0.614")
print("   Random Forest:    F1=0.571 | AUC=0.539 | Acc=0.560")

print("\nOTIMIZADO (Após tuning):")
if xgb_tuned is not None:
    xgb_final = pull() if 'xgb_results' in locals() else "N/A"
    print(f"   XGBoost Tuned:    [Métricas acima]")
    
if gbc_tuned is not None:
    print(f"   GBC Tuned:        [Métricas acima]")
    
if rf_tuned is not None:
    print(f"   RF Tuned:         [Métricas acima]")
    
if ensemble_voting is not None:
    print(f"   Ensemble:         [Métricas acima]")

# ===============================================================
# 12. PRÓXIMOS PASSOS
# ===============================================================

print(f"\n🚀 PRÓXIMOS PASSOS")
print("=" * 60)
print("1. ✅ Modelos otimizados e prontos")
print("2. 📊 Evaluação detalhada com evaluate_model()")
print("3. 🎯 Finalização com finalize_model()")
print("4. 🧪 Teste no dataset holdout (TestDS.csv)")
print("5. 📋 Interpretabilidade e feature importance")
print("6. 💾 Salvar modelos finais")

print(f"\n🎉 OTIMIZAÇÃO CONCLUÍDA!")
print("💡 Use evaluate_model(modelo) para análise detalhada")
print("💾 Use finalize_model(modelo) para treino no dataset completo")